In [1]:
from google_agent import Google_agent
from deep_research import Deep_research_engine
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
from pydantic import Field
from google.oauth2.credentials import Credentials
from langchain_google_genai import ChatGoogleGenerativeAI
import nest_asyncio
nest_asyncio.apply()
from langchain_openai import ChatOpenAI

import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
pse=os.getenv('pse')
openai_api_key=os.getenv('openai_api_key')
tavily_client = TavilyClient(api_key=tavily_key)
composio_api_key=os.getenv('composio_api_key')
pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))
GEMINI_MODEL='gemini-2.0-flash'
langchain_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model=GEMINI_MODEL, temperature=0.3)


In [2]:
from token_creator import get_creds

In [3]:
creds=get_creds()

In [4]:

# if os.path.exists("token.json"):
#     creds = Credentials.from_authorized_user_file("token.json")
# creds=Credentials.from_authorized_user_info(info={'refresh_token':"",
#                                                   'client_id':"",
#                                                   'client_secret':""},
#                                                   scopes=["https://mail.google.com/", "https://www.googleapis.com/auth/calendar", "https://www.googleapis.com/auth/cloud-platform", "https://www.googleapis.com/auth/contacts", "https://www.googleapis.com/auth/tasks", "https://www.googleapis.com/auth/generative-language.retriever"])
api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'creds':creds,
    'openai_api_key':openai_api_key,
    'composio_key':composio_api_key
}


In [5]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

@dataclass
class Deps:
    deep_research_output: dict

In [12]:
class Cortana_agent:
    def __init__(self, api_keys:dict):
        """
        Args:
            
            api_keys (dict): The API keys to use as a dictionary
        """
        GEMINI_MODEL='gemini-2.0-flash'
        
        pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=api_keys['google_api_key']))
        openai_llm=ChatOpenAI(api_key=api_keys['openai_api_key'])
        langchain_llm = ChatGoogleGenerativeAI(google_api_key=api_keys['google_api_key'], model=GEMINI_MODEL, temperature=0.3)
        # tools
        llms={'pydantic_llm':pydantic_llm,
              'langchain_llm':langchain_llm,
              'openai_llm':openai_llm}
        deep_research_engine=Deep_research_engine(pydantic_llm,api_keys)
        
        def google_agent_tool(query:str):
            """
            Use this tool to interact with the google agent, which can, search for images, manage the user's calendar, emails, google tasks, contacts, and more.
            Args:
                query (str): The entire query related to the google agent and its capabilities
            Returns:
                str: The response from the google agent
            """
            google_agent=Google_agent(llms,api_keys)
            try:
                res=google_agent.chat(query)
                return res
            except Exception as e:
                return f"An error occurred: {e}"
       

        async def search_and_question_answering_tool(ctx: RunContext[Deps], query:str):
            """
            Use this tool to do a deep research on a topic, to gather detailed informations and data, answer_questions from the deep research results or do a quick research if the answer is not related to the deep research.
            Args:
                query (str): The query related to the search_and_question_answering_tool and its capabilities
                

            Returns:
                str: The response from the search_and_question_answering_tool
            """
            @dataclass
            class Route:
                answer: str = Field(default_factory=None,description="the answer to the question if the question is related to the deep research")
                route: str = Field(description="the route, either deep_research or answer_question, or quick_research")
            agent=Agent(pydantic_llm, output_type=Route, instructions="you are a router/question answering agent, you are given a query and you need to decide what to do based on the information provided")
            response= agent.run_sync(f"based on the query: {query}, and the information provided: {ctx.deps.deep_research_output if ctx.deps.deep_research_output else ''} either answer the question or if the answer is not related to the information provided or need more information return 'quick_research' or 'deep_research'")
            route=response.output.route
            if route=='deep_research':
                response=deep_research_engine.chat(query)
                ctx.deps.deep_research_output=response
                return response
            elif route=='answer_question':
                return response.output.answer
            elif route=='quick_research':
                quick_research_agent=Agent(pydantic_llm, tools=[tavily_search_tool(api_keys['tavily_key'])], instructions="do a websearch based on the query")
                result= quick_research_agent.run_sync(query)
                return result.output

        def get_current_time_tool():
            """
            Use this tool to get the current time.
            Returns:
                str: The current time in a formatted string
            """
        
            return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        @dataclass
        class cortana_output:
            full_answer: str = Field(description="the full answer to the user's query with output from the tools if tools are used")
            voice_answer: str = Field(description="if the answer is longer than 100 words, return a shorter answer to the user's query for the voice assistant else return the full answer")
        self.agent=Agent(pydantic_llm, output_type=cortana_output, tools=[google_agent_tool, search_and_question_answering_tool, get_current_time_tool], instructions="you are Cortana, a helpful assistant that can help with a wide range of tasks,\
                          you can use the tools provided to you to help the user with their queries, ask how you can help the user")
        self.memory=Message_state(messages=[])
        self.deps=Deps(deep_research_output={})
    def chat(self, query:str):
        result=self.agent.run_sync(query, deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.output


In [13]:
cortana_agent=Cortana_agent(api_keys)

In [14]:
res=cortana_agent.chat('send a picture of a cat to padioutristan@gmail.com')
print(res)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions


  warnings.warn(


Cortana_agent.__init__.<locals>.cortana_output(full_answer='I have sent a picture of a cat to padioutristan@gmail.com', voice_answer='I have sent a picture of a cat to padioutristan@gmail.com')


In [9]:
cortana_agent.memory.messages

[]